# ta-lib-003 ( Feature‑Engineering + Sequence Models)
https://chatgpt.com/c/68092b32-ba28-800a-b3c1-35ba49d501ad

In [ ]:
import numpy as np
import pandas as pd
from talib import RSI, MACD, BBANDS, ATR
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Attention, concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
#  Load OHLCV data from CSV. Expects columns: ['DATE','TIME','OPEN','HIGH','LOW','CLOSE','TICKVOL','VOL','SPREAD']
def load_data(csv_path):
    df = pd.read_csv("../datasets/XAGUSD-H1-rates.csv", parse_dates=[['DATE', 'TIME']])
    df.set_index('DATE_TIME', inplace=True)
    return df

In [ ]:
# Compute TA indicators and append to dataframe.
def compute_indicators(df):
    df = df.copy()
    df['rsi'] = RSI(df['CLOSE'], timeperiod=14)
    macd, macd_signal, macd_hist = MACD(df['CLOSE'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['macd_hist'] = macd_hist
    upper, middle, lower = BBANDS(df['CLOSE'], timeperiod=20)
    df['bb_z'] = (df['CLOSE'] - middle) / (upper - lower)
    df['atr'] = ATR(df['HIGH'], df['LOW'], df['CLOSE'], timeperiod=14)
    df.dropna(inplace=True)
    return df

In [ ]:
# Standard scale selected feature columns. Returns scaled array and scaler object.
def scale_features(df, feature_cols):
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df[feature_cols])
    return scaled, scaler

In [ ]:
# Label reversal points: 1 for reversal start, 0 otherwise. Here: local max followed by lookahead bearish move > threshold.
def label_reversals(df, lookahead=5, threshold=0.001):
    close = df['CLOSE'].values
    labels = np.zeros(len(close))
    for i in range(1, len(close) - lookahead - 1):
        if close[i] > close[i - 1] and close[i] > close[i + 1]:
            future_min = close[i + 1:i + 1 + lookahead].min()
            if (close[i] - future_min) / close[i] > threshold:
                labels[i] = 1
        if close[i] < close[i - 1] and close[i] < close[i + 1]:
            future_max = close[i + 1:i + 1 + lookahead].max()
            if (future_max - close[i]) / close[i] > threshold:
                labels[i] = 2  # sell reversal
    return labels.astype(int)

In [ ]:
# Build sliding-window sequences and corresponding labels.
def create_sequences(features, labels, window_size=60):
    X, y = [], []
    for i in range(window_size, len(features)):
        X.append(features[i - window_size:i])
        y.append(labels[i])
    return np.array(X), np.array(y)

In [ ]:
# Build LSTM + attention reversal classifier.
def build_model(input_shape):
    inp = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(inp)
    x = Dropout(0.2)(x)
    # attention
    attn = Attention()([x, x])
    x = LSTM(32)(attn)
    x = Dropout(0.2)(x)
    out = Dense(3, activation='softmax')(x)
    model = Model(inp, out)
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Example usage:
# df = load_data('XAGUSD-H1-rates.csv')\# compute indicators
# df_ind = compute_indicators(df)
# features, scaler = scale_features(df_ind, ['rsi','macd_hist','bb_z','atr'])
# labels = label_reversals(df_ind)
# X, y = create_sequences(features, labels)
# model = build_model(X.shape[1:])
